# **CUHK-STAT3009** Notebook - Matrix Factorization


In [18]:
# Load and pro-processed dataset
import numpy as np
import pandas as pd

## Upload Netflix dataset in CUHK-STAT3009 Github repo

train_url = "https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/train.csv"
test_url = "https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/test.csv"

dtrain = pd.read_csv(train_url)
dtest = pd.read_csv(test_url)

train_rating = dtrain['rating'].values
train_rating = np.array(train_rating, dtype=float)
train_pair = dtrain[['user_id', 'movie_id']].values

test_rating = dtest['rating'].values
test_rating = np.array(test_rating, dtype=float)
test_pair = dtest[['user_id', 'movie_id']].values

n_user = max( max(train_pair[:,0]), max(test_pair[:,0]) ) + 1
n_item = max( max(train_pair[:,1]), max(test_pair[:,1]) ) + 1

def rmse(test_rating, pred_rating):
    return np.sqrt( np.mean( (pred_rating - test_rating)**2) )

## ALS-MF

### Idea
  - BCD perfectly fits our formulation
  - solve $\mathbf{Q}$ (fixed $\mathbf{P}$) $\to$ solve $\mathbf{P}$ (fixed $\mathbf{Q}$) $\to$ ...

### Steps

- When $\mathbf{P}$ is fixed, the objective function for $\mathbf{Q}$ is a standard QP, and each $\mathbf{q}_i$ can be solved **parallelly** with an **analytic solution**:
$$ \mathbf{q}^{(l+1)}_i = ( \sum_{u \in \mathcal{U}_i} \mathbf{p}^{(l)}_u (\mathbf{p}^{(l)}_u)^\intercal + \lambda |\Omega| \mathbf{I})^{-1} \sum_{u \in \mathcal{U}_i} r_{ui} \mathbf{p}^{(l)}_u $$

- When $\mathbf{Q}$ is fixed, the objective function for $\mathbf{P}$ is a standard QP, and each $\mathbf{p}_i$ can be solved **parallelly** with an **analytic solution**.
$$\mathbf{p}^{(l+1)}_u =  ( \sum_{i \in \mathcal{I}_u} \mathbf{q}^{(l+1)}_i (\mathbf{q}^{(l+1)}_i)^\intercal + \lambda |\Omega| \mathbf{I})^{-1} \sum_{i \in \mathcal{I}_u} r_{ui} \mathbf{q}^{(l+1)}_i$$


### illustrated with user-0 and item-0

In [3]:
## Initialization
K, lam = 5, .1 # hps
n_obs = len(train_rating)
P = np.random.randn(n_user, K)
Q = np.random.randn(n_item, K)

index_item = [np.where(train_pair[:,1] == i)[0] for i in range(n_item)]
index_user = [np.where(train_pair[:,0] == u)[0] for u in range(n_user)]

In [4]:
## illustrated with user-0 and item-0
# update q0
item_id = 0
index_item_tmp = index_item[item_id]
# compute `sum_pu` and `sum_matrix`
sum_pu, sum_matrix = np.zeros((K)), np.zeros((K, K))
for record_ind in index_item_tmp:
  user_id, rating_tmp = train_pair[record_ind][0], train_rating[record_ind]
  print('users with item-0: %s' %user_id)
  sum_matrix = sum_matrix + np.outer(P[user_id,:], P[user_id,:])
  sum_pu = sum_pu + rating_tmp * P[user_id,:]

users with item-0: 425
users with item-0: 161
users with item-0: 566


In [6]:
Q[item_id,:] = np.dot(np.linalg.inv(sum_matrix + lam*n_obs*np.identity(K)), sum_pu)
print('the one-step sol of q0: %s' %Q[item_id,:])

the one-step sol of q0: [ 4.07894574e-05 -3.05766755e-04  1.27221346e-03 -5.28671369e-04
  3.98681667e-04]


In [7]:
# update p0
user_id = 0
index_user_tmp = index_user[user_id]
# compute `sum_qi` and `sum_matrix`
sum_qi, sum_matrix = np.zeros((K)), np.zeros((K, K))
for record_ind in index_user_tmp:
  item_id, rating_tmp = train_pair[record_ind][1], train_rating[record_ind]
  print('item with user-0: %s' %item_id)
  sum_matrix = sum_matrix + np.outer(Q[item_id,:], Q[item_id,:])
  sum_qi = sum_qi + rating_tmp * Q[item_id,:]

item with user-0: 2720
item with user-0: 1935
item with user-0: 3126
item with user-0: 3386
item with user-0: 1118
item with user-0: 1686
item with user-0: 151
item with user-0: 1456
item with user-0: 2893
item with user-0: 571
item with user-0: 712
item with user-0: 586
item with user-0: 3148
item with user-0: 2306
item with user-0: 3295
item with user-0: 395
item with user-0: 2198
item with user-0: 1546
item with user-0: 1164
item with user-0: 3256
item with user-0: 426
item with user-0: 977
item with user-0: 2885
item with user-0: 1019
item with user-0: 3487
item with user-0: 1292
item with user-0: 3383
item with user-0: 1577
item with user-0: 2790
item with user-0: 2337
item with user-0: 2345
item with user-0: 945
item with user-0: 1428
item with user-0: 1306
item with user-0: 2977
item with user-0: 2531
item with user-0: 1133
item with user-0: 1778
item with user-0: 1473
item with user-0: 3276
item with user-0: 2157
item with user-0: 1501
item with user-0: 993
item with user-0: 35

In [8]:
P[user_id,:] = np.dot(np.linalg.inv(sum_matrix + lam*n_obs*np.identity(K)), sum_qi)
print('the one-step sol of q0: %s' %P[user_id,:])

the one-step sol of q0: [ 0.00664027 -0.00767711 -0.00038677  0.00286579 -0.00149553]


### Make a `skikit-learn` compatible class


- **Parameter**:
    - #Users: `n`
    - #Items: `m`
    - latent factors for users: `P`
    - latent factors for items: `Q`
    - l2-weight (*hp*): `lam`
    - #Latent factors (*hp*): `K`

- **Method**:

  - `fit`: 
    - *input*: `train_pair`, `train_rating`
    - *output*: fitted P and Q
  - `predict`: 
    - *input*: `test_pair`
    - *output*: predicted ratings
  - `rmse`: 
    - *input*: `test_pair`, `test_rating`
    - *output*: RMSE for the predicted ratings

  - `obj`: 
    - *input*: `test_pair`, `test_rating`
    - *output*: objective function for the MF method


In [16]:
class MF(object):

    def __init__(self, n_user, n_item, lam=.001, K=10, iterNum=50, tol=1e-4, verbose=1):
        self.P = np.random.randn(n_user, K)
        self.Q = np.random.randn(n_item, K)
        # self.index_item = []
        # self.index_user = []
        self.n_user = n_user
        self.n_item = n_item
        self.lam = lam
        self.K = K
        self.iterNum = iterNum
        self.tol = tol
        self.verbose = verbose

    def fit(self, train_pair, train_rating):
        diff, tol = 1., self.tol
        n_user, n_item, n_obs = self.n_user, self.n_item, len(train_pair)
        K, iterNum, lam = self.K, self.iterNum, self.lam
        ## store user/item index set
        self.index_item = [np.where(train_pair[:,1] == i)[0] for i in range(n_item)]
        self.index_user = [np.where(train_pair[:,0] == u)[0] for u in range(n_user)]
        
        if self.verbose:
            print('Fitting Reg-MF: K: %d, lam: %.5f' %(K, lam))
        
        for i in range(iterNum):
            ## item update
            obj_old = self.obj(test_pair=train_pair, test_rating=train_rating)
            for item_id in range(n_item):
                index_item_tmp = self.index_item[item_id]
                if len(index_item_tmp) == 0:
                    self.Q[item_id,:] = 0.
                    continue
                ## compute `sum_pu` and `sum_matrix`
                sum_pu, sum_matrix = np.zeros((K)), np.zeros((K, K))
                for record_ind in index_item_tmp:
                    ## double-check
                    if item_id != train_pair[record_ind][1]:
                        raise ValueError('the item_id is worning in updating Q!')
                    user_id, rating_tmp = train_pair[record_ind][0], train_rating[record_ind]
                    sum_matrix = sum_matrix + np.outer(self.P[user_id,:], self.P[user_id,:])
                    sum_pu = sum_pu + rating_tmp * self.P[user_id,:]                    
                self.Q[item_id,:] = np.dot(np.linalg.inv(sum_matrix + lam*n_obs*np.identity(K)), sum_pu)
            
            for user_id in range(n_user):
                index_user_tmp = self.index_user[user_id]
                if len(index_user_tmp) == 0:
                    self.P[user_id,:] = 0.
                    continue
                ## compute `sum_qi` and `sum_matrix`
                sum_qi, sum_matrix = np.zeros((K)), np.zeros((K, K))
                for record_ind in index_user_tmp:
                    ## double-check
                    if user_id != train_pair[record_ind][0]:
                        raise ValueError('the user_id is worning in updating P!')
                    item_id, rating_tmp = train_pair[record_ind][1], train_rating[record_ind]
                    sum_matrix = sum_matrix + np.outer(self.Q[item_id,:], self.Q[item_id,:])
                    sum_qi = sum_qi + rating_tmp * self.Q[item_id,:]                    
                self.P[user_id,:] = np.dot(np.linalg.inv(sum_matrix + lam*n_obs*np.identity(K)), sum_qi)
            # compute the new rmse score
            obj_new = self.obj(test_pair=train_pair, test_rating=train_rating)
            diff = abs(obj_new - obj_old) / obj_old
            if self.verbose:
                print("Reg-MF: ite: %d; diff: %.3f Obj: %.3f" %(i, diff, obj_new))
            if(diff < tol):
                break

    def predict(self, test_pair):
        # predict ratings for user-item pairs
        pred_rating = [np.dot(self.P[line[0]], self.Q[line[1]]) for line in test_pair]
        return np.array(pred_rating)
    
    def rmse(self, test_pair, test_rating):
        # report the rmse for the fitted `MF`
        pred_rating = self.predict(test_pair=test_pair)
        return np.sqrt( np.mean( (pred_rating - test_rating)**2) )
      
    def obj(self, test_pair, test_rating):
        return (self.rmse(test_pair, test_rating))**2 + self.lam*np.sum(self.P**2) + self.lam*np.sum(self.Q**2)

In [20]:
# fitting
cue = MF(n_user, n_item, K=5, lam=.0001)
cue.fit(train_pair=train_pair, train_rating=train_rating)

Fitting Reg-MF: K: 5, lam: 0.00010
Reg-MF: ite: 0; diff: 0.559 Obj: 9.882
Reg-MF: ite: 1; diff: 0.750 Obj: 2.475
Reg-MF: ite: 2; diff: 0.117 Obj: 2.186
Reg-MF: ite: 3; diff: 0.054 Obj: 2.068
Reg-MF: ite: 4; diff: 0.035 Obj: 1.997
Reg-MF: ite: 5; diff: 0.024 Obj: 1.949
Reg-MF: ite: 6; diff: 0.016 Obj: 1.917
Reg-MF: ite: 7; diff: 0.011 Obj: 1.896
Reg-MF: ite: 8; diff: 0.007 Obj: 1.883
Reg-MF: ite: 9; diff: 0.004 Obj: 1.875
Reg-MF: ite: 10; diff: 0.003 Obj: 1.870
Reg-MF: ite: 11; diff: 0.002 Obj: 1.866
Reg-MF: ite: 12; diff: 0.001 Obj: 1.864
Reg-MF: ite: 13; diff: 0.001 Obj: 1.863
Reg-MF: ite: 14; diff: 0.000 Obj: 1.862
Reg-MF: ite: 15; diff: 0.000 Obj: 1.862
Reg-MF: ite: 16; diff: 0.000 Obj: 1.861
Reg-MF: ite: 17; diff: 0.000 Obj: 1.861
Reg-MF: ite: 18; diff: 0.000 Obj: 1.861
Reg-MF: ite: 19; diff: 0.000 Obj: 1.861


In [21]:
# pediction
pred_rating = cue.predict(test_pair)
pred_train_rating = cue.predict(train_pair)
print('train rmse: %.3f; test rmse: %.3f' %(rmse(train_rating, pred_train_rating), rmse(test_rating, pred_rating)))

train rmse: 0.854; test rmse: 1.189


### Make a `skikit-learn` compatible class for baseline methods

| Method |  Model                   | param                            |
|--------| ----------------------   |----------------------------------|
| Glb    | $\hat{r}_{ui}$ = $\mu_0$ |$\mu_0$                           |
|User    | $\hat{r}_{ui}$ = $a_u$   |$\mathbf{a}$ = ($a_1$, ..., $a_n$)|
| Item   | $\hat{r}_{ui}$ = $b_i$   |$\mathbf{b}$ = ($b_1$, ..., $b_m$)|

In [23]:
class glb_mean(object):
	def __init__(self):
		self.glb_mean = 0
	
	def fit(self, train_ratings):
		self.glb_mean = np.mean(train_ratings)
	
	def predict(self, test_pair):
		pred = np.ones(len(test_pair))
		pred = pred*self.glb_mean
		return pred

class user_mean(object):
	def __init__(self, n_user):
		self.n_user = n_user
		self.glb_mean = 0.
		self.user_mean = np.zeros(n_user)
	
	def fit(self, train_pair, train_ratings):
		self.glb_mean = train_ratings.mean()
		for u in range(self.n_user):
			ind_train = np.where(train_pair[:,0] == u)[0]
			if len(ind_train) == 0:
				self.user_mean[u] = self.glb_mean
			else:
				self.user_mean[u] = train_ratings[ind_train].mean()
	
	def predict(self, test_pair):
		pred = np.ones(len(test_pair))*self.glb_mean
		j = 0
		for row in test_pair:
			user_tmp, item_tmp = row[0], row[1]
			pred[j] = self.user_mean[user_tmp]
			j = j + 1
		return pred

class item_mean(object):
	def __init__(self, n_item):
		self.n_item = n_item
		self.glb_mean = 0.
		self.item_mean = np.zeros(n_item)
	
	def fit(self, train_pair, train_ratings):
		self.glb_mean = train_ratings.mean()
		for i in range(self.n_item):
			ind_train = np.where(train_pair[:,1] == i)[0]
			if len(ind_train) == 0:
				self.item_mean[i] = self.glb_mean
			else:
				self.item_mean[i] = train_ratings[ind_train].mean()
	
	def predict(self, test_pair):
		pred = np.ones(len(test_pair))*self.glb_mean
		j = 0
		for row in test_pair:
			user_tmp, item_tmp = row[0], row[1]
			pred[j] = self.item_mean[item_tmp]
			j = j + 1
		return pred

## Sequential modeling: baseline models + MF

In [24]:
## Baseline + LFM
# glb mean
glb_ave = glb_mean()
glb_ave.fit(train_rating)
pred = glb_ave.predict(test_pair)

# user_mean
train_rating_cm = train_rating - glb_ave.predict(train_pair)
user_ave = user_mean(n_user=n_user)
user_ave.fit(train_pair=train_pair, train_ratings=train_rating_cm)
train_rating_res = train_rating_cm - user_ave.predict(train_pair)
pred = pred + user_ave.predict(test_pair)

# fit MF RS by residual ratings 
cue = MF(n_user, n_item, K=5, lam=.0001)
cue.fit(train_pair=train_pair, train_rating=train_rating_res)
pred = pred + cue.predict(test_pair)

print('RMSE for glb + user_mean + MF: %.3f' %rmse(test_rating, pred) )

Fitting Reg-MF: K: 5, lam: 0.00010
Reg-MF: ite: 0; diff: 0.895 Obj: 0.923
Reg-MF: ite: 1; diff: 0.110 Obj: 0.822
Reg-MF: ite: 2; diff: 0.058 Obj: 0.774
Reg-MF: ite: 3; diff: 0.021 Obj: 0.758
Reg-MF: ite: 4; diff: 0.010 Obj: 0.750
Reg-MF: ite: 5; diff: 0.005 Obj: 0.747
Reg-MF: ite: 6; diff: 0.003 Obj: 0.744
Reg-MF: ite: 7; diff: 0.002 Obj: 0.743
Reg-MF: ite: 8; diff: 0.001 Obj: 0.742
Reg-MF: ite: 9; diff: 0.001 Obj: 0.741
Reg-MF: ite: 10; diff: 0.001 Obj: 0.740
Reg-MF: ite: 11; diff: 0.001 Obj: 0.740
Reg-MF: ite: 12; diff: 0.001 Obj: 0.739
Reg-MF: ite: 13; diff: 0.000 Obj: 0.739
Reg-MF: ite: 14; diff: 0.000 Obj: 0.739
Reg-MF: ite: 15; diff: 0.000 Obj: 0.739
Reg-MF: ite: 16; diff: 0.000 Obj: 0.738
Reg-MF: ite: 17; diff: 0.000 Obj: 0.738
Reg-MF: ite: 18; diff: 0.000 Obj: 0.738
Reg-MF: ite: 19; diff: 0.000 Obj: 0.738
Reg-MF: ite: 20; diff: 0.000 Obj: 0.738
Reg-MF: ite: 21; diff: 0.000 Obj: 0.738
Reg-MF: ite: 22; diff: 0.000 Obj: 0.737
Reg-MF: ite: 23; diff: 0.000 Obj: 0.737
Reg-MF: ite: 24

## To-do list

- **STAT**
  - [ ] Idea and movitation of Matrix factorization (MF)
  - [ ] Idea of Blockwise coordinate descent (BCD)
  - [ ] Use BCD to solve regularized MF
  - [ ] Able to find a solution of BCD to a general QP
  - [ ] Use MF to make prediction for RS

- **Code**
  - [ ] Param and hps of MF
  - [ ] Implement MF by using MF
  - [ ] Implement BCD for a general QP